#### Importamos librerias

In [1]:
from pycaret.classification import *
import pandas as pd
from sklearn.metrics import classification_report

#ver todas las columnas
pd.set_option('display.max_columns', None)

#### Cargamos y preparamos datos

In [2]:
df = pd.read_csv('datos_procesados//accidentes_procesados.csv')
df

,Expediente,Fecha,Día semana,Tipo día,Hora,Tramo horario,Distrito,Tipo de vía,Calle,Latitud,Longitud,Tipo accidente,Estado meteorológico,Tipo vehiculo,Implicado,Sexo,Edad,Lesividad,Positivo alcohol,Positivo droga
0,2018S017842,2019-02-04,lunes,Laborable,09:10:00,Mañana,Chamberí,Calle,Alberto Aguilera,40.429592,-3.706555,Colisión,Despejado,Motocicleta,Conductor,Hombre,40-49,Leve,0,0.0
1,2018S017842,2019-02-04,lunes,Laborable,09:10:00,Mañana,Chamberí,Calle,Alberto Aguilera,40.429592,-3.706555,Colisión,Despejado,Turismo,Conductor,Mujer,30-39,Leve,0,0.0
2,2019S000001,2019-01-01,martes,Festivo,03:45:00,Madrugada,Carabanchel,Calle,Antonio De Leyva,40.385903,-3.717037,Colisión,Despejado,Turismo,Conductor,Hombre,40-49,Muy leve,1,0.0
3,2019S000001,2019-01-01,martes,Festivo,03:45:00,Madrugada,Carabanchel,Calle,Antonio De Leyva,40.385903,-3.717037,Colisión,Despejado,Turismo,Conductor,Mujer,40-49,Muy leve,0,0.0
4,2019S000001,2019-01-01,martes,Festivo,03:45:00,Madrugada,Carabanchel,Calle,Antonio De Leyva,40.385903,-3.717037,Colisión,Despejado,Turismo,Conductor,Mujer,40-49,Muy leve,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238102,2024S015161,2024-04-30,martes,Laborable,22:47:00,Noche,Salamanca,Calle,Rufino Blanco,40.426675,-3.665416,Colisión,Despejado,Turismo,Conductor,Hombre,18-29,Leve,0,0.0
238103,2024S015162,2024-04-30,martes,Laborable,23:18:00,Noche,Salamanca,Calle,Rufino Blanco,40.426675,-3.665416,Colisión,Nublado,Vehiculo Pesado,Conductor,Hombre,40-49,Muy leve,0,0.0
238104,2024S015162,2024-04-30,martes,Laborable,23:18:00,Noche,Salamanca,Calle,Rufino Blanco,40.426675,-3.665416,Colisión,Nublado,Turismo,Conductor,Hombre,50-59,Muy leve,0,0.0
238105,2024S015164,2024-04-30,martes,Laborable,23:15:00,Noche,Fuencarral - El Pardo,Avenida,Monforte De Lemos,40.479788,-3.702582,Otro,Despejado,Turismo,Conductor,Hombre,18-29,Grave,0,0.0


#### Configuramos entorno

In [5]:
setup(data=df, target='Lesividad', session_id=357)

,Description,Value
0,Session id,357
1,Target,Lesividad
2,Target type,Multiclass
3,Target mapping,"Fallecido: 0, Grave: 1, Leve: 2, Muy leve: 3"
4,Original data shape,"(238107, 20)"
5,Transformed data shape,"(238107, 96)"
6,Transformed train set shape,"(166674, 96)"
7,Transformed test set shape,"(71433, 96)"
8,Numeric features,4
9,Categorical features,15


#### Comparamos modelos

In [6]:
best_model = compare_models() # Comparar modelos y seleccionar el mejor

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7812,0.7515,0.7812,0.7444,0.7560,0.3463,0.3587,13.1020
et,Extra Trees Classifier,0.7572,0.7745,0.7572,0.7234,0.7359,0.2902,0.2977,15.5720
dummy,Dummy Classifier,0.7553,0.5000,0.7553,0.5705,0.6500,0.0000,0.0000,2.2830
ridge,Ridge Classifier,0.7393,0.0000,0.7393,0.6518,0.6805,0.1005,0.1215,2.5180
rf,Random Forest Classifier,0.7385,0.7254,0.7385,0.7140,0.7240,0.2707,0.2732,10.1670
svm,SVM - Linear Kernel,0.7295,0.0000,0.7295,0.7049,0.7057,0.2333,0.2413,5.6100
gbc,Gradient Boosting Classifier,0.7188,0.0000,0.7188,0.6990,0.7079,0.2316,0.2328,62.8190
lightgbm,Light Gradient Boosting Machine,0.7151,0.5999,0.7151,0.6995,0.7065,0.2321,0.2328,4.6450
lda,Linear Discriminant Analysis,0.7116,0.0000,0.7116,0.6847,0.6964,0.1923,0.1944,4.8090
lr,Logistic Regression,0.7078,0.0000,0.7078,0.6701,0.6855,0.1488,0.1526,21.4410


In [7]:
best_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

#### Creamos el modelo

In [8]:
knn = create_model('knn')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7787,0.7502,0.7787,0.7408,0.7535,0.3398,0.3519
1,0.7795,0.7448,0.7795,0.7406,0.7526,0.3356,0.3493
2,0.7817,0.7517,0.7817,0.7461,0.7568,0.3483,0.3606
3,0.7829,0.7502,0.7829,0.7477,0.7578,0.3502,0.3631
4,0.7811,0.7547,0.7811,0.7440,0.7561,0.3480,0.3598
5,0.7851,0.7578,0.7851,0.7500,0.7611,0.3618,0.3735
6,0.7789,0.7460,0.7789,0.7402,0.7519,0.3326,0.3466
7,0.7815,0.7556,0.7815,0.7469,0.7576,0.3502,0.3619
8,0.7830,0.7543,0.7830,0.7454,0.7577,0.3514,0.3641


#### Afinamos el modelo

In [9]:
tune_model_knn = tune_model(knn, optimize='F1') # Afinar el modelo

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7951,0.7967,0.7951,0.7538,0.7628,0.3646,0.3862
1,0.7936,0.7857,0.7936,0.7544,0.7601,0.3547,0.3781
2,0.7932,0.7964,0.7932,0.7523,0.7608,0.3591,0.3803
3,0.7952,0.7880,0.7952,0.7603,0.7637,0.3653,0.3869
4,0.7931,0.7909,0.7931,0.7521,0.7620,0.3632,0.3826
5,0.7985,0.7953,0.7985,0.7612,0.7686,0.3817,0.4013
6,0.7901,0.7891,0.7901,0.7534,0.7564,0.3409,0.3649
7,0.7965,0.7978,0.7965,0.7581,0.7651,0.3701,0.3916
8,0.7934,0.7954,0.7934,0.7519,0.7610,0.3582,0.3799


Fitting 10 folds for each of 10 candidates, totalling 100 fits


La puntuación F1 es una métrica que combina la precisión y el recall del modelo. Es útil cuando tienes un conjunto de datos desbalanceado, es decir, cuando una clase es mucho más frecuente que la otra

#### Evaluamos el modelo

In [13]:
evaluate_model(tune_model_knn) 

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

#### Predecimos con el modelo

In [14]:
predict_model(tune_model_knn) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.7923,0.7933,0.7923,0.7522,0.7606,0.3580,0.3784


#### Guardamos el modelo

In [ ]:
modelo_final = finalize_model(tune_model_knn) # Guardar el modelo 

In [ ]:
# Definir la ruta del directorio 'modelos'
carpeta_modelos = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'modelos')

# Verificar si el directorio 'modelos' no existe; de ser así, crearlo
if not os.path.exists(carpeta_modelos):
    os.makedirs(carpeta_modelos)

# Guardar el modelo
save_model(modelo_final, os.path.join(carpeta_modelos, 'modelo_lesividad'))

In [17]:
modelo_final # Mostrar el modelo final

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Latitud', 'Longitud',
                                             'Positivo alcohol',
                                             'Positivo droga'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_f...
                                                              min_samples_leaf=20,
                                                              return_df=True,
                                                              smoothing=10,
                                                              verbose=0))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                ('actual_estimator',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=-1, n_neighbors=19, p=2,
                                      weights='distance'))],
         verbose=False)

In [18]:
y_true = df['Lesividad']
y_pred = df['prediction_label']

# Generar el reporte de clasificación
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

   Fallecido       0.00      0.00      0.00        45
       Grave       0.33      0.04      0.08      3611
        Leve       0.55      0.37      0.44     13824
    Muy leve       0.83      0.95      0.89     53953

    accuracy                           0.79     71433
   macro avg       0.43      0.34      0.35     71433
weighted avg       0.75      0.79      0.76     71433



Exactitud (Accuracy): La exactitud general del modelo es 0.80, lo que significa que aproximadamente el 80% de las predicciones realizadas son correctas en todo el conjunto de datos.

En resumen, el modelo parece tener un rendimiento razonablemente bueno para las clases "Leve" y "Muy leve", pero un rendimiento muy pobre para las clases "Fallecido" y "Grave" ya que hay muy pocos casos.